In [153]:
import pandas as pd
import folium
from folium.plugins import HeatMap


In [154]:
#Load the Abatement File
currentabatement = pd.read_csv('CurrentAbatements.csv')
#Load the land use dictionary
landuse = pd.read_csv('land_use_dict.csv')
landuse.head(5)

,CATEGORY,DESCRIPTION,DEPARTMENT OF TAX EQUALIZATION,HAMILTON COUNTY
0,AGRICULTURAL,VACANT LAND,100.0,100
1,AGRICULTURAL,CASH - GRAIN OR GENERAL FARM,101.0,101
2,AGRICULTURAL,LIVESTOCK FARM EXCEPT DAIRY & POULTRY,102.0,102
3,AGRICULTURAL,DAIRY FARMS,103.0,103
4,AGRICULTURAL,POULTRY FARMS,104.0,104


In [155]:
#Create a dictionary for land use codes
description = landuse.DESCRIPTION.tolist()
codes = landuse['HAMILTON COUNTY'].tolist()
res = {codes[i]: description[i] for i in range(len(description))} 

#Map the values onto the abatement dataframe
currentabatement['land use english'] = currentabatement['land use'].map(res)

In [156]:
#See how many of each type of abatement there is
currentabatement['land use english'].value_counts()

SINGLE FAMILY                                2975
CONDOMINIUMS                                 1328
P.U.D. (LANDOMINIUM)                          128
VACANT LAND                                    93
CONDOMINIUM OFFICE BUILDING                    70
TWO FAMILY DWELLINGS                           66
APARTMENTS - 40+ UNITS                         43
RETAIL - APARTMENTS OVER                       35
THREE FAMILY DWELLINGS                         35
APARTMENTS - 4 TO 19 UNITS                     30
WAREHOUSE                                      26
OFFICE, ELEVATOR (3 STORIES PLUS)              23
LOW INCOME HOUSE TAX CREDIT(commercial)        23
LIGHT MANUFACTURING                            20
PARKING GARAGES / LOTS                         13
OFFICE BUILDINGS                               13
OTHER STRUCTURES                               12
APARTMENTS - 20 TO 39 UNITS                    11
SMALL DETACHED RETAIL (10,000)                 11
LOW INCOME HOUSE TAX CREDIT (residential)       9


In [157]:
#Filter all of the single family properties out of the list
single_family = currentabatement[currentabatement['land use'] == 510]
#Check how many failed to geocode
fails = single_family.latitude.isnull().sum()
#out of how many total
total = len(single_family)
#completion rate
rate = 100 - round((fails / total), 4) * 100
print('Currently Geocoded: {}%'.format(rate))

Currently Geocoded: 80.2%


In [158]:
#Drop all of the properties that failed to geocode
single_family = single_family.dropna(axis=0, subset=['latitude', 'longitude'])
#See how many single family abatements there are in the dataframe
len(single_family)

2386

In [159]:
#Normalize the abatement value to a value between 0 and 1
x, y = single_family.abatement_value.min(), single_family.abatement_value.max()
single_family['abatement_normal']=(single_family.abatement_value-x)/(y-x)

In [160]:
#Establish the map
map1 = folium.Map(
    location=[39.1255405,-84.476745],
    tiles='cartodbpositron',
    zoom_start=12,
)
#Iterate over the dataset and create coordinates and weight [latitude, longitude, normalized abatement value]
data = []
for x in range(len(single_family)):
    data.append([single_family.latitude.iloc[x], single_family.longitude.iloc[x], single_family.abatement_normal.iloc[x]])

#Apply the data and set the radius/blur of the heatmap    
HeatMap(data, radius=8, blur=5,).add_to(map1)
#Display the map
map1